In [2]:
import kagglehub
import numpy as np
import pandas as pd

from tabulate import tabulate
import matplotlib.pyplot as plt

from wordcloud import WordCloud
import seaborn as sns
from textblob import TextBlob
from googletrans import Translator

from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

from langdetect import detect, DetectorFactory

import string
import re
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Ensure NLTK resources are downloaded
nltk.download('stopwords')
nltk.download('wordnet')

/Users/suprita/Desktop/data lab/DataScienceLabFinalProject/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/suprita/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/suprita/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# validated
path = kagglehub.dataset_download("saurabhshahane/fake-news-classification")

print("Path to dataset files:", path)

# load datasets
df1=pd.read_csv(path+'/WELFake_Dataset.csv')
df1['article'] = df1['title'] + df1['text']
df1 = df1[['article', 'label']]
df1.head()

Path to dataset files: /Users/suprita/.cache/kagglehub/datasets/saurabhshahane/fake-news-classification/versions/77


,article,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,1
1,NaN,1
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,1
3,"Bobby Jindal, raised Hindu, uses story of Chri...",0
4,SATAN 2: Russia unvelis an image of its terrif...,1


In [4]:
# validated
path = kagglehub.dataset_download("jillanisofttech/fake-or-real-news")
df2 = pd.read_csv(path + '/fake_or_real_news.csv')
df2['label'] = df2['label'].map({'FAKE':0, 'REAL': 1})
df2['article'] = df2['title'] + df2['text']
df2 = df2[['article', 'label']]
df2.head()

,article,label
0,"You Can Smell Hillary’s FearDaniel Greenfield,...",0
1,Watch The Exact Moment Paul Ryan Committed Pol...,0
2,Kerry to go to Paris in gesture of sympathyU.S...,1
3,Bernie supporters on Twitter erupt in anger ag...,0
4,The Battle of New York: Why This Primary Matte...,1


In [5]:
# validated
path = kagglehub.dataset_download("clmentbisaillon/fake-and-real-news-dataset")
df3_1, df3_2 = pd.read_csv(path + '/Fake.csv'), pd.read_csv(path + '/True.csv')
df3_1['label'], df3_2['label'] = 0, 1
df3 = pd.concat([df3_1, df3_2])
df3 = df3.sample(frac=1, random_state=42).reset_index(drop=True)
df3['article'] = df3['title'] + df3['text']
df3 = df3[['article', 'label']]
df3.head()

,article,label
0,Ben Stein Calls Out 9th Circuit Court: Committ...,0
1,Trump drops Steve Bannon from National Securit...,1
2,Puerto Rico expects U.S. to lift Jones Act shi...,1
3,OOPS: Trump Just Accidentally Confirmed He Le...,0
4,Donald Trump heads for Scotland to reopen a go...,1


In [ ]:
# albanian source
# validated

df4 = pd.read_csv('DataPreprocessing/alb_to_eng_allfakes.csv')
df4.head()

In [153]:
# DetectorFactory.seed = 42  # For consistent results

# # Function to detect language
# def detect_language(text):
#     try:
#         return detect(text)
#     except Exception as e:
#         print(f"Error detecting language: {text[:30]}... - {e}")
#         return None
    

# # Drop rows still in Albanian
# df4['language'] = df4['article'].apply(detect_language)
# df4 = df4[df4['language'] == 'en']  # Keep only rows detected as English

# # Drop the language column after filtering
# df4.drop(columns=['language'], inplace=True)

# # Display the cleaned DataFrame
# df4.head()

,article,label
0,"Whoever visits Ulcinj, here's what happens to ...",0
1,Vucic: From Lavrov I received disturbing infor...,1
2,"NASA's discovery, here's what there used to be...",1
3,"In Montenegro, 52 new cases with Covid-19 in M...",1
4,"Gani Geci: VV are Communists, they must ""disap...",0


In [155]:
# df4.to_csv('alb_to_eng_allfakes.csv', index=False)

In [160]:
# covid19 fake news tweets
# validated

path = kagglehub.dataset_download("elvinagammed/covid19-fake-news-dataset-nlp")
df5 = pd.read_csv(path + '/english_test_with_labels.csv')
df5['label'] = df5['label'].map({'fake':0, 'real':1})
df5['article'] = df5['tweet']
df5 = df5[['article', 'label']]
df5.head()

,article,label
0,Our daily update is published. States reported...,1
1,Alfalfa is the only cure for COVID-19.,0
2,President Trump Asked What He Would Do If He W...,0
3,States reported 630 deaths. We are still seein...,1
4,This is the sixth time a global health emergen...,1


In [161]:
# validated

path = kagglehub.dataset_download("ronikdedhia/fake-news")
df6= pd.read_csv(path + '/fake_news.csv')
df6['article'] = df6['title'] + df6['text']
df6['label'] = df6['label'].map({0:1, 1:0})
df6 = df6[['article', 'label']]
df6.head()

,article,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,0
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",1
2,Why the Truth Might Get You FiredWhy the Truth...,0
3,15 Civilians Killed In Single US Airstrike Hav...,0
4,Iranian woman jailed for fictional unpublished...,0


In [162]:
# covid headlines of fake news, taking out
# validated

path = kagglehub.dataset_download("thesumitbanik/covid-fake-news-dataset")
df7 = pd.read_csv(path+ '/data.csv')
df7['article'] = df7['headlines']
df7['label'] = df7['outcome']
df7 = df7[['article', 'label']]
df7.head()

,article,label
0,A post claims compulsory vacination violates t...,0
1,A photo claims that this person is a doctor wh...,0
2,Post about a video claims that it is a protest...,0
3,All deaths by respiratory failure and pneumoni...,0
4,The dean of the College of Biologists of Euska...,0


In [130]:
df1.shape, df2.shape, df3.shape, df4.shape

((72134, 2), (6335, 2), (44898, 2), (3882, 2))

In [131]:
df5.shape, df6.shape, df7.shape

((2140, 2), (20800, 2), (10201, 2))

In [187]:
df = pd.concat([df1, df2, df3, df4, df5, df6, df7])
df.shape

(159275, 2)

In [188]:
df.dropna(inplace=True)
df.drop_duplicates(subset='article', inplace=True)
df.shape

(76769, 2)

In [189]:
df.head()

,article,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,1
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,1
3,"Bobby Jindal, raised Hindu, uses story of Chri...",0
4,SATAN 2: Russia unvelis an image of its terrif...,1
5,About Time! Christian Group Sues Amazon and SP...,1


In [190]:
df['sentiment_polarity'] = df['article'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)
df['sentiment_subjectivity'] = df['article'].apply(lambda x: TextBlob(str(x)).sentiment.subjectivity)

In [167]:
df.head()

,article,label,sentiment_polarity,sentiment_subjectivity
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,1,0.034641,0.296824
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,1,0.190402,0.512798
3,"Bobby Jindal, raised Hindu, uses story of Chri...",0,0.111493,0.387106
4,SATAN 2: Russia unvelis an image of its terrif...,1,0.013342,0.403577
5,About Time! Christian Group Sues Amazon and SP...,1,-0.100461,0.428571


# preprocessing

In [191]:
X_train, X_test, y_train, y_test = train_test_split(df['article'], df['label'], test_size=0.2, random_state=42, stratify=df['label'])
overlap = set(X_train).intersection(set(X_test))
print(f"Number of overlapping samples between train and test sets: {len(overlap)}")

Number of overlapping samples between train and test sets: 0


In [192]:
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    text = re.sub(r'\b([a-zA-Z])\.(?=[a-zA-Z])', r'\1specialdot', text)  # Preserve acronyms
    text = re.sub(r'\b([a-zA-Z])\.(?=\s|$)', r'\1specialdot', text)
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = text.replace("specialdot", ".")
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenize, remove stopwords, and lemmatize
    tokens = text.split()
    stop_words = set(stopwords.words('english'))
    stop_words.discard("not")
    stop_words.discard("no")
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# Vectorize text
def vectorize(text, maxFeats, ngram):
    vectorizer = TfidfVectorizer(max_features=maxFeats, ngram_range=ngram, stop_words='english')
    X_tfidf = vectorizer.fit_transform(text)
    return X_tfidf, vectorizer

# Load data
X_train, X_test, y_train, y_test = train_test_split(
    df['article'], df['label'], 
    test_size=0.2, random_state=42, stratify=df['label']
)

# Preprocess data
X_train_processed = X_train.apply(preprocess_text)
X_test_processed = X_test.apply(preprocess_text)

# Vectorize data
X_train_vectorized, vectorizer = vectorize(X_train_processed, 2000, (1,1))
X_test_vectorized = vectorizer.transform(X_test_processed)

# Train model
model = RandomForestClassifier()
model.fit(X_train_vectorized, y_train)

# Predict and evaluate on test set
y_pred = model.predict(X_test_vectorized)
print(classification_report(y_test, y_pred))
print(f"Test Set Accuracy: {accuracy_score(y_test, y_pred)}")

# # Cross-validation
# scores = cross_val_score(model, X_train_vectorized, y_train, cv=5, scoring='accuracy')
# print("Cross-validation Accuracies:", scores)
# print("Average Accuracy:", scores.mean())


              precision    recall  f1-score   support

           0       0.93      0.93      0.93      9097
           1       0.89      0.89      0.89      6257

    accuracy                           0.91     15354
   macro avg       0.91      0.91      0.91     15354
weighted avg       0.91      0.91      0.91     15354

Test Set Accuracy: 0.9140940471538361


# testing

In [170]:
# Evaluate on new test data
test_data = [
    {"article": "Breaking news: The President announces a major economic reform to boost GDP growth.", "label": 1},
    {"article": "Scientists claim that the Earth is flat and provide evidence at a recent conference.", "label": 0},
    {"article": "Recent studies confirm the safety and effectiveness of COVID-19 vaccines worldwide.", "label": 1},
    {"article": "A man in Ohio claims he saw an alien spaceship while walking his dog late at night.", "label": 0},
    {"article": "The government has approved new environmental regulations to tackle climate change.", "label": 1},
    {"article": "Fake story circulates on social media claiming that sugar cures all diseases.", "label": 0},
    {"article": "The international summit on renewable energy has successfully concluded in Paris.", "label": 1},
    {"article": "A viral post claims that drinking only water for a month can cure cancer.", "label": 0},
    {"article": "Breaking: NASA confirms the discovery of a new habitable planet in a nearby star system.", "label": 1},
    {"article": "Authorities warn about a rise in fake news websites spreading misinformation online.", "label": 1}
]

# Convert to DataFrame
test_data_df = pd.DataFrame(test_data)

# Preprocess and vectorize test data
test_data_processed = test_data_df['article'].apply(preprocess_text)
test_data_vectorized = vectorizer.transform(test_data_processed)

# Predict labels for test data
y_pred_test = model.predict(test_data_vectorized)

# Evaluate model predictions
print("Predictions vs Ground Truth:")
for text, pred, true in zip(test_data_df['article'], y_pred_test, test_data_df['label']):
    print(f"News: {text}\nPrediction: {'True' if pred == 1 else 'Fake'} | Ground Truth: {'True' if true == 1 else 'Fake'}\n")

# Classification report for test data
print(classification_report(test_data_df['label'], y_pred_test, target_names=["Fake", "True"]))

Predictions vs Ground Truth:
News: Breaking news: The President announces a major economic reform to boost GDP growth.
Prediction: Fake | Ground Truth: True

News: Scientists claim that the Earth is flat and provide evidence at a recent conference.
Prediction: Fake | Ground Truth: Fake

News: Recent studies confirm the safety and effectiveness of COVID-19 vaccines worldwide.
Prediction: Fake | Ground Truth: True

News: A man in Ohio claims he saw an alien spaceship while walking his dog late at night.
Prediction: Fake | Ground Truth: Fake

News: The government has approved new environmental regulations to tackle climate change.
Prediction: Fake | Ground Truth: True

News: Fake story circulates on social media claiming that sugar cures all diseases.
Prediction: Fake | Ground Truth: Fake

News: The international summit on renewable energy has successfully concluded in Paris.
Prediction: Fake | Ground Truth: True

News: A viral post claims that drinking only water for a month can cure can

/Users/suprita/Desktop/data lab/DataScienceLabFinalProject/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/suprita/Desktop/data lab/DataScienceLabFinalProject/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/suprita/Desktop/data lab/DataScienceLabFinalProject/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divisi

In [171]:
# Download latest version
path = kagglehub.dataset_download("bhavikjikadara/fake-news-detection")

print("Path to dataset files:", path)

# load datasets
fake=pd.read_csv(path+'/fake.csv')
fake['label'] = 0
true=pd.read_csv(path+'/true.csv')
true['label'] = 1
test = pd.concat([fake, true])
test.head()

Path to dataset files: /Users/suprita/.cache/kagglehub/datasets/bhavikjikadara/fake-news-detection/versions/1


,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [172]:
test['article'] = test['title'] + test['text']
test = test[['article', 'label']]

In [173]:
test.dropna(inplace=True)
test.drop_duplicates(subset='article', inplace=True)
test.shape

(39105, 2)

In [176]:
test['sentiment_polarity'] = test['article'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)
test['sentiment_subjectivity'] = test['article'].apply(lambda x: TextBlob(str(x)).sentiment.subjectivity)
test.head()

,article,label,sentiment_polarity,sentiment_subjectivity
0,Donald Trump Sends Out Embarrassing New Year’...,0,0.083136,0.597204
1,Drunk Bragging Trump Staffer Started Russian ...,0,-0.020811,0.344802
2,Sheriff David Clarke Becomes An Internet Joke...,0,-0.012345,0.541969
3,Trump Is So Obsessed He Even Has Obama’s Name...,0,-0.038021,0.413021
4,Pope Francis Just Called Out Donald Trump Dur...,0,-0.011722,0.495222


In [177]:
test = test.sample(frac=1, random_state=42).reset_index(drop=True)
test.head()

,article,label,sentiment_polarity,sentiment_subjectivity
0,Many 'lost' voters say they have found their c...,1,0.079703,0.430014
1,"Iranian Revolutionary Guards commander, anothe...",1,0.034635,0.199219
2,Cambodian leader gets China's backing as West ...,1,0.047220,0.297890
3,Ex-KKK Grand Wizard: Maine Gov. Right To Call...,0,-0.036366,0.372324
4,FINGER-WAGGING GRANNY HILLARY CLINTON Blows Up...,0,0.350000,0.650000


In [ ]:
X, y = test['article'], test['label']
X = X.apply(preprocess_text)
X = vectorizer.fit_transform(X)
# Predict and evaluate on test set
y_pred = model.predict(X)
print(classification_report(y, y_pred))
print(f"Test Set Accuracy: {accuracy_score(y, y_pred)}")

In [59]:
pd.Series(y_pred).value_counts()

1    36104
0     3001
Name: count, dtype: int64